In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import dgl.data
# 一个DGL Dataset可能包含一个或多个graph。 但是这里的Cora dataset 只包含一个graph
dataset = dgl.data.CoraGraphDataset()
# print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [5]:
print('Number of categories:', dataset.num_classes)

Number of categories: 7


In [8]:
# 查看dataset的属性有哪些
dataset.__dict__

{'_reverse_edge': True,
 '_name': 'cora_v2',
 '_url': 'https://data.dgl.ai/dataset/cora_v2.zip',
 '_force_reload': False,
 '_verbose': True,
 '_hash_key': (),
 '_hash': 'f9065fa7',
 '_raw_dir': '/home/lawson/.dgl',
 '_save_dir': '/home/lawson/.dgl',
 '_g': Graph(num_nodes=2708, num_edges=10556,
       ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={}),
 '_graph': <networkx.classes.digraph.DiGraph at 0x7f214c34ec10>,
 '_num_classes': 7}

In [14]:
g=dataset._g # 拿到这个数据集对应的图
print(g)

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


In [17]:
"""
根据输出可以看到ndata 是个什么样数据。
(1)dict
(2)feat即feature, label， test_mask, train_mask,val_mask 等都是什么？
"""
print(g.ndata) # 打印节点特征
print("----")
print(g.edata) # 打印边特征

{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False])}
----
{}


In [22]:
# 查看一下ndata 的属性
print(type(g.ndata)) 
label = g.ndata['label']
print(label.size()) # 这就意味着有2708篇文档的分类结果

<class 'dgl.view.HeteroNodeDataView'>
torch.Size([2708])


In [ ]:
'''
如果g是一个单节点类型的图，...
如果g是一个多节点类型的图，g.ndata[feat] 返回的是一个dict，dict中的格式是{str:tensor}。映射节点类型到节点特征中。 
'''

In [23]:
from dgl.nn import GraphConv # 直接从DGL中引入GCN

"""
如果想构建一个多层GCN，那么简单的堆叠 dlg.nn.GraphConv 模块就可以了（这个模块继承自 troch.nn.Module），像下面这么实现：
"""
class GCN(nn.Module):
    # in_feats: input features
    # h_feats : hidden features
    # num_classes : num of class
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)  # 看一下这个传入的参数代表的含义
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [20]:
"""
定义一个训练函数
"""
def train(g, model):
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    print(features)
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features) # 输入features

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

# 使用CPU进行训练
# model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
# train(g, model)


# 将模型放到GPU上
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]],
       device='cuda:0')
In epoch 0, loss: 1.946, val acc: 0.236 (best 0.236), test acc: 0.217 (best 0.217)
In epoch 5, loss: 1.897, val acc: 0.552 (best 0.574), test acc: 0.535 (best 0.592)
In epoch 10, loss: 1.825, val acc: 0.624 (best 0.624), test acc: 0.656 (best 0.656)
In epoch 15, loss: 1.731, val acc: 0.662 (best 0.662), test acc: 0.684 (best 0.684)
In epoch 20, loss: 1.610, val acc: 0.678 (best 0.678), test acc: 0.703 (best 0.703)
In epoch 25, loss: 1.467, val acc: 0.686 (best 0.686), test acc: 0.714 (best 0.714)
In epoch 30, loss: 1.306, val acc: 0.720 (best 0.720), test acc: 0.735 (best